# Mixed and HDG methods

In [19]:
from ngsolve import *
from ngsolve.webgui import Draw


## Model problem

Poisson equation 
\begin{equation}
-\nabla \cdot (c \nabla u) = f, \quad u = u_{d}, \text{ on } \Gamma_{D}, \quad c\nabla u \cdot n = g, \text{ on }\Gamma_N  
\end{equation}

In [20]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
source = sin(3.14*x)
ud = CF(5)
g = mesh.BoundaryCF( {"left" : y*(1-y)}, default=0)
c = 10

## Primal formulation and method

In [10]:
fesp = H1(mesh, order=4, dirichlet="bottom")
up, vp = fesp.TnT()

ap = BilinearForm(lam*grad(up)*grad(vp)*dx).Assemble()
fp = LinearForm(source*vp*dx + g*vp * ds).Assemble()

gfup = GridFunction(fesp)
gfup.Set(ud, BND)

r = fp.vec - ap.mat * gfup.vec
gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * r
Draw (-1*lam * grad(gfup), mesh, "flux-primal");
Draw (gfup)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Mixed formulation and method
\begin{equation}
-\nabla \cdot (c \nabla u) = f, \quad u = u_{d}, \text{ on } \Gamma_{D}, \quad c\nabla u \cdot n = g, \text{ on }\Gamma_N  
\end{equation}

\begin{equation}
a q = \nabla u, \quad \nabla \cdot q = f, \quad u = u_{d}, \text{ on } \Gamma_{D}, \quad -q\cdot n = g, \text{ on }\Gamma_N  
\end{equation}

In [9]:
order_flux=1
V = HDiv(mesh, order=order_flux, dirichlet="right|top|left")
Q = L2(mesh, order=order_flux-1)
fesm = V*Q
q, u = fesm.TrialFunction()
r, v = fesm.TestFunction()
normal = specialcf.normal(mesh.dim)

afun = 1/lam
am = BilinearForm((afun*q*r - div(q)*v - div(r)*u)*dx).Assemble()
fm = LinearForm(-source*v*dx - ud*(r.Trace()*normal)*ds).Assemble()

gfm = GridFunction(fesm)

gfq, gfu = gfm.components
gfq.Set(-g*normal, BND)
res = fm.vec.data - am.mat * gfm.vec
gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * res
# solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfsigma, mesh, "flux-mixed")
Draw (gfu, mesh, "u-mixed");

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

## HDG formulation and method 

$$ \widehat{q}_h\cdot n = q_h\cdot n + \tau(u_h - \widehat{u}_h)$$

\begin{align*}
\int_{K} a q_h \cdot r - \int_{K} u_h\nabla\cdot r +\int_{\partial K} \widehat{u}_h r\cdot n ds &= 0 \\
\int_{K} q_h \cdot\nabla v + \int_{\partial K} \widehat{q}_h\cdot n v ds &= -\int_{K} f v \\
\sum_{K}\int_{\partial K \backslash \Gamma_D} \widehat{q}_h\cdot n \mu & = \int_{\Gamma_N} g \mu ds \\
\int_{\Gamma_D} \widehat{u}_h \mu ds & = \int_{Gamma_D} u_d \mu ds
\end{align*}

In [16]:
p=1; condense=False
V = VectorL2(mesh, order=p)
W = L2(mesh, order=p)
M = FacetFESpace(mesh, order=p, dirichlet='bottom')
feshdg = V*W*M
q, u, uhat = feshdg.TrialFunction()
r, v, mu   = feshdg.TestFunction()
normal = specialcf.normal(mesh.dim)
# stabilization parameter
tau = 1.0
ahdg = BilinearForm(feshdg, condense=condense)
ahdg += (afun*q*r - u*div(r))*dx + uhat*r*normal*dx(element_boundary=True)
ahdg += -div(q)*v*dx + (u-uhat)*v*dx(element_boundary=True)
ahdg += (q*normal+tau*(u-uhat))*mu*dx(element_boundary=True)
ahdg.Assemble()
fhdg = LinearForm(-source*v*dx - g*mu.Trace()*ds).Assemble()


gfhdg = GridFunction(feshdg)
gfq, gfu, gfuhat = gfhdg.components
gfuhat.Set(ud, BND)
res = fhdg.vec.data - ahdg.mat * gfhdg.vec
gfhdg.vec.data += ahdg.mat.Inverse(freedofs=feshdg.FreeDofs(), inverse="umfpack") * res
# solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfq, mesh, "flux-hdg")
Draw (gfu, mesh, "u-hdg");

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…